In [1]:
import pandas as pd
import numpy as np

In [38]:
import copy
from copy import deepcopy

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [210]:
from tqdm import tqdm_notebook as tqdm

In [59]:
pd.set_option('display.max_rows', 200)

In [4]:
df = pd.read_csv('final_df.csv')

In [8]:
list(df.columns)

['ticker',
 'investment_strategy',
 'equity_style',
 'fixed_income_style',
 'broad_asset class',
 'broad_category',
 'category_name',
 'prospectus_objective',
 'fund_family',
 'benchmark1',
 'div_yield_ttm_real',
 '30_day_sec_yield_real',
 'weighted_avg_pe_ratio_real',
 'weighted_avg_price_to_sales_ratio',
 'weighted_avg_price_to_book_ratio',
 'weighted_median_ROE',
 'weighted_median_roa',
 'forecasted_div_yield_real',
 'forecasted_pe_ratio',
 'forecasted_price_to_sales_ratio',
 'forecasted_price_to_book_ratio',
 'nu_of_holdings',
 'forecasted_ps_ratio',
 'stock_forecasted_price_to_book_value',
 'stock_number_of_stock_holdings',
 'bonds_current_yield',
 'bond_yield_to_maturity',
 'effective_duration',
 'avg_coupon',
 'avg_credit_score_1_best',
 'bond_average_price',
 'bond_number_of_holdings',
 'forecasted_5yr_earnings_growth',
 'forecasted_book_value_growth',
 'forecasted_cash_flow_growth',
 'forecasted_earnings_growth',
 'forecasted_revenue_growth',
 'region_exposure_americas',
 'reg

In [12]:
df.select_dtypes(exclude=['float64','int64']).columns

Index(['investment_strategy', 'equity_style', 'fixed_income_style',
       'broad_asset class', 'broad_category', 'category_name',
       'prospectus_objective', 'fund_family', 'benchmark1',
       'open_to_new_investors_y', 'fee_12b1', 'inception_date', 'index_fund_y',
       'leverage_y', 'fund_of_funds_y', 'fund_currency_hedged_y',
       'fund_socially_responsible_y', 'fund_available_for_insurance_place_y',
       'comparables_1', 'comparables_2', 'comparables_3', 'comparables_4',
       'comparables_5', 'expense_ratio', 'turnover_ratio', 'holdings',
       'new_holdings'],
      dtype='object')

In [180]:
df['holdings']

ticker
VFIAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
FXAIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VGTSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VITSX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSMX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTPSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VIIIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VINIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VBTLX    ['federal_national_mortgage_association_3%', '...
FCNTX    ['facebook_inc_a', 'amazon.com_inc', 'berkshir...
VWENX    ['microsoft_corp', 'verizon_communications_inc...
VTIAX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
DODGX    ['wells_fargo_&_co', 'charles_schwab_corp', 'o...
PIMIX    ['irs_usd_3.00000_06/19/19-5y_cme', 'irs_jpy_0...
RERGX    ['airbus_se', 'aia_group_ltd', 'nintendo_co_lt...
VWIUX    ['southeast_ala_gas_supply_dist_gas_supp

In [35]:
df.select_dtypes(exclude=['float64','int64']).head()

,investment_strategy,equity_style,fixed_income_style,broad_asset class,broad_category,category_name,prospectus_objective,fund_family,benchmark1,fee_12b1,inception_date,comparables_1,comparables_2,comparables_3,comparables_4,comparables_5,holdings,new_holdings
ticker,,,,,,,,,,,,,,,,,,
VFIAX,The investment seeks to track the performance ...,Large Cap/Blend,--,US Equity,Equity,Large Blend,Growth,Vanguard,S&P 500 TR USD 100.0%,--,Inception Date 11/13/2000,TISPX TIAA-CREF S&P 500 Index Instl,GEQYX GuideStone Funds Equity Index Instl,SPIIX SEI S&P 500 Index I (SIMT),DFUSX DFA US Large Company I,VINIX Vanguard Institutional Index I,"['microsoft_corp', 'apple_inc', 'amazon.com_in...",microsoft_corp apple_inc amazon.com_inc facebo...
VTSAX,The investment seeks to track the performance ...,Large Cap/Blend,--,US Equity,Equity,Large Blend,Growth,Vanguard,CRSP US Total Market TR USD 100.0%,--,Inception Date 11/13/2000,VLISX Vanguard Large Cap Index Institutional,PLFIX Principal Large Cap S&P 500 Index Inst,VITPX Vanguard Instl Ttl Stk Mkt Idx InstlPls,TISPX TIAA-CREF S&P 500 Index Instl,VFIAX Vanguard 500 Index Admiral,"['microsoft_corp', 'apple_inc', 'amazon.com_in...",microsoft_corp apple_inc amazon.com_inc facebo...
FXAIX,The investment seeks to provide investment res...,Large Cap/Blend,--,US Equity,Equity,Large Blend,Growth and Income,Fidelity Investments,S&P 500 TR USD 100.0%,--,Inception Date 5/4/2011,SSEYX State Street Equity 500 Index II,SWPPX Schwab® S&P 500 Index,SWTSX Schwab Total Stock Market Index,USPRX USAA 500 Index Reward,BSPPX iShares S&P 500 Index Investor P,"['microsoft_corp', 'apple_inc', 'amazon.com_in...",microsoft_corp apple_inc amazon.com_inc facebo...
VGTSX,The investment seeks to track the performance ...,Large Cap/Blend,--,International Equity,Equity,Foreign Large Blend,Foreign Stock,Vanguard,FTSE Global All Cap ex US (USA) NR USD 100.0%,--,Inception Date 4/29/1996,VDVIX Vanguard Developed Markets Index Inv,ARTKX Artisan International Value Investor,FSPSX Fidelity® International Index,FSGGX Fidelity® Global ex US Index,None,"['nestle_sa', 'alibaba_group_holding_ltd_adr',...",nestle_sa alibaba_group_holding_ltd_adr taiwan...
VITSX,The investment seeks to track the performance ...,Large Cap/Blend,--,US Equity,Equity,Large Blend,Growth,Vanguard,CRSP US Total Market TR USD 100.0%,--,Inception Date 7/7/1997,VLISX Vanguard Large Cap Index Institutional,PLFIX Principal Large Cap S&P 500 Index Inst,VITPX Vanguard Instl Ttl Stk Mkt Idx InstlPls,TISPX TIAA-CREF S&P 500 Index Instl,VFIAX Vanguard 500 Index Admiral,"['microsoft_corp', 'apple_inc', 'amazon.com_in...",microsoft_corp apple_inc amazon.com_inc facebo...


In [ ]:
df['turnover_ratio'] = df['turnover_ratio'].str.extract('(\-?\d+.\d+)').astype(float)

df['fund_available_for_insurance_place_y'] = df['fund_available_for_insurance_place_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [18]:
df['open_to_new_investors_y'] = df['open_to_new_investors_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [21]:
df['index_fund_y'] =df['index_fund_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [23]:
df['fund_currency_hedged_y'] = df['fund_currency_hedged_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [24]:
df['fund_socially_responsible_y']=df['fund_socially_responsible_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [25]:
df['fund_available_for_insurance_place_y'] = df['fund_available_for_insurance_place_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [27]:
df['fund_of_funds_y'] = df['fund_of_funds_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [28]:
df['leverage_y'] = df['leverage_y'].apply(lambda x: 1 if x.endswith('Yes') else 0)

In [32]:
df['turnover_ratio'] = df['turnover_ratio'].str.extract('(\-?\d+.\d+)').astype(float)

In [34]:
df['expense_ratio'] = df['expense_ratio'].str.extract('(\-?\d+.\d+)').astype(float)

In [ ]:
'investment_strategy', 'equity_style', 'fixed_income_style', 'broad_asset class', 'broad_category', 'category_name', 'prospectus_objective', 'fund_family', 'benchmark1', 'fee_12b1', 'inception_date', 'comparables_1', 'comparables_2', 'comparables_3', 'comparables_4', 'comparables_5', 'holdings', 'new_holdings'

In [39]:
df1 = deepcopy(df.drop(columns={'investment_strategy', 'equity_style', 'fixed_income_style', 'broad_asset class', 'broad_category', 'category_name', 'prospectus_objective', 'fund_family', 'benchmark1', 'fee_12b1', 'inception_date', 'comparables_1', 'comparables_2', 'comparables_3', 'comparables_4', 'comparables_5', 'holdings', 'new_holdings'}))

In [40]:
df1.to_csv('final_df1.csv')

In [41]:
import numpy as np; import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [154]:
df1.set_index('ticker', inplace=True)

In [86]:
cosine_similarity(df1)

array([[1.        , 0.96513145, 0.96533471, ..., 0.96873017, 0.99309599,
        0.96126458],
       [0.96513145, 1.        , 0.87059743, ..., 0.87032891, 0.93171034,
        0.86034556],
       [0.96533471, 0.87059743, 1.        , ..., 0.98984967, 0.97578895,
        0.97651607],
       ...,
       [0.96873017, 0.87032891, 0.98984967, ..., 1.        , 0.98870496,
        0.99721196],
       [0.99309599, 0.93171034, 0.97578895, ..., 0.98870496, 1.        ,
        0.9866074 ],
       [0.96126458, 0.86034556, 0.97651607, ..., 0.99721196, 0.9866074 ,
        1.        ]])

In [169]:
similarity_matrix = cosine_similarity(df1)

In [170]:
def recommendations(name, similarity_matrix = similarity_matrix):
    
    recommended_hotels = []
    
    # gettin the index of the hotel that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(similarity_matrix[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(df1.index)[i])
        
    return recommended_hotels

In [171]:
recomendations = input("Give a ticker for a fund you own ")
recommendations(recomendations)

Give a ticker for a fund you own MFLDX


['HFMDX',
 'DDFLX',
 'MTPKX',
 'TSCIX',
 'FIODX',
 'GHTIX',
 'CUHCX',
 'DHSIX',
 'RPIBX',
 'JCUIX']

In [172]:
df2.loc[recommendations(recomendations)][['fund_name','expense_ratio','avg_credit_score_1_best','risk_beta','risk_sortino_ratio','total_return_5y','risk_monthly_VAR','stock_sector_rising_interest_rate_bad']]

,fund_name,expense_ratio,avg_credit_score_1_best,risk_beta,risk_sortino_ratio,total_return_5y,risk_monthly_VAR,stock_sector_rising_interest_rate_bad
ticker,,,,,,,,
HFMDX,Hennessy Cornerstone Mid Cap 30 Inv,1.31,0.0,0.9840,0.5353,3.07,18.24,17.25
DDFLX,Delaware Floating Rate Instl,0.69,15.0,-0.1343,0.3046,2.95,1.77,0.00
MTPKX,Manning & Napier Target 2030 K,1.00,8.0,0.5471,0.3559,4.84,12.59,26.84
TSCIX,AMG TimesSquare Small Cap Growth Z,0.98,0.0,1.0750,0.5771,10.01,16.45,21.28
FIODX,Delaware Intl Opportunities Bd Inst,1.08,9.0,0.7643,-0.3540,-0.59,4.97,0.00
GHTIX,Leland Real Asset Opportuns I,1.61,0.0,0.8610,-0.2669,-0.21,10.49,68.87
CUHCX,Victory US 500 Enh Vol Wtd Idx C,1.74,0.0,0.9231,1.0600,7.24,8.73,23.88
DHSIX,Diamond Hill Small Cap Fund I,0.98,0.0,0.9251,0.6391,4.16,15.36,13.34
RPIBX,T. Rowe Price Intl Bond Fd,0.67,12.0,1.0830,0.7639,1.50,5.42,0.00


In [168]:
df1.drop(columns= {'risk_score_low_bond_quality', 'risk_score_below_B_over_15p', 'risk_score_yield_2_maturity_low', 'risk_score_duration_high', 'risk_score_preferred_net_over25p', 'stock_sector_rising_interest_rate_bad', 'risk_score_bad_equities_rising_rates', 'risk_score_avg_price_book_high', 'risk_score_avg_price_earnings_high', 'risk_score_smallest_companies', 'risk_score_smallest_companies_over50', 'risk_score_emerging_markets_over_45', 'risk_score_energy_over_15', 'risk_total', 'risk_score_PE_expensive', 'risk_score_PS_expensive', 'risk_score_low_ROE', 'risk_score_duration_long', 'risk_score_low_bond_yields', 'risk_score_risky_bonds', 'risk_score_latam'}, inplace=True)


In [96]:
indices = pd.Series(df1.index)

In [109]:
df1.head()

,div_yield_ttm_real,30_day_sec_yield_real,weighted_avg_pe_ratio_real,weighted_avg_price_to_sales_ratio,weighted_avg_price_to_book_ratio,weighted_median_ROE,weighted_median_roa,forecasted_div_yield_real,forecasted_pe_ratio,forecasted_price_to_sales_ratio,forecasted_price_to_book_ratio,nu_of_holdings,bonds_current_yield,bond_yield_to_maturity,effective_duration,avg_coupon,avg_credit_score_1_best,bond_average_price,forecasted_5yr_earnings_growth,forecasted_book_value_growth,forecasted_cash_flow_growth,forecasted_earnings_growth,forecasted_revenue_growth,region_exposure_americas,region_exposure_north_america,region_exposure_latin_america,region_exposure_greater_europe,region_exposure_uk,region_exposure_europe_developed,region_exposure_europe_emerging,region_exposure_africa_middle_east,region_exposure_greater_asia,region_exposure_japan,region_exposure_australiasia,region_exposure_asia_developed,region_exposure_asia_emerging,region_exposure_developed_markets,region_exposure_emerging_markets,stock_sector_exp_basic_materials,stock_sector_exp_communication_service,stock_sector_exp_consumer_cycl,stock_sector_exp_cons_defensive,stock_sector_exp_energy,stock_sector_exp_fin_serv,stock_sector_exp_healthcare,stock_sector_exp_industrial,stock_sector_exp_real_estate,stock_sector_exp_tech,stock_sector_exp_utilities,stock_market_cap_giant,stock_market_cap_large,stock_market_cap_medium,stock_market_cap_small,stock_market_cap_micro,risk_beta,risk_sharpe_ratio,risk_sortino_ratio,risk_daily_VAR,risk_monthly_VAR,stock_style_exp_lcvalue,stock_style_exp_lcblend,stock_style_exp_lcgrowth,stock_style_exp_mcvalue,stock_style_exp_mcblend,stock_style_exp_bcgrowth,stock_style_exp_scvalue,stock_style_exp_smblend,stock_style_exp_smgrowth,open_to_new_investors_y,min_initial_investment,fee_actual_mgmt_fee,index_fund_y,leverage_y,fund_of_funds_y,fund_currency_hedged_y,fund_socially_responsible_y,fund_available_for_insurance_place_y,total_return_1m,total_return_3m,total_return_6m,total_return_ytd,total_return_1y,total_return_3y,total_return_5y,total_return_10y,total_return_15y,total_return_20y,bond_quality_AAA,bond_quality_AA,bond_quality_A,bond_quality_BBB,bond_quality_BB,bond_quality_B,bond_quality_below_B,position_cash_net,position_cash_long,position_cash_short,position_stock_net,position_stock_long,position_stock_short,position_bond_net,position_bond_long,position_bond_short,position_convertible_net,position_convertible_long,position_convertible_short,position_preferred_net,position_preferred_long,position_preferred_short,position_other_net,position_other_long,position_other_short,expense_ratio,aum,share_aum,net_asset_value,turnover_ratio,fund_flows,risk_score_low_bond_quality,risk_score_below_B_over_15p,risk_score_yield_2_maturity_low,risk_score_duration_high,risk_score_preferred_net_over25p,stock_sector_rising_interest_rate_bad,risk_score_bad_equities_rising_rates,risk_score_avg_price_book_high,risk_score_avg_price_earnings_high,risk_score_smallest_companies,risk_score_smallest_companies_over50,risk_score_emerging_markets_over_45,risk_score_energy_over_15,risk_total,risk_score_PE_expensive,risk_score_PS_expensive,risk_score_low_ROE,risk_score_duration_long,risk_score_low_bond_yields,risk_score_risky_bonds,risk_score_latam,00,000,0000,00000,000000_maturity_20250612,000000_maturity_20250920,000000_maturity_20260126,000000_maturity_20281115,00000_,00000_0,00000_02,00000_03,00000_06,00000_07,00000_08,00000_09,00000_10yr_tnotes_dec9,00000_11,00000_112495,00000_112498,00000_112501_fee_ccpcdx,00000_112504_fee_ccpcdx,00000_112505_fee_ccpitraxx,00000_112507_fee_ccpcdx,00000_112513_fee_ccpcdx,00000_112519_fee_ccpcdx,00000_112524_fee_ccpcdx,00000_12,00000_1_ccpcdx,00000_1_ccpitraxx,00000_2_ccpcdx,00000_2i65brsb7_fee_ccpcdx,00000_2i65bybd6_fee_ccpcdx,00000_2i65bzdm1_fee_ccpcdx,00000_558600990_fee_ccpcdx,00000_96569886_fee_ccpitraxx,00000_99d0b4bz9_ccpvanilla,00000_cdx_ccp_ice,00000_cdxig033_fee_ccpcdx,00000_cdxnahy_500bp_ccp,00000_cdxnaig_100bp_ccp,00000_cosp9u3205_fee_c

In [84]:
df1.isna().sum().sort_values()

div_yield_ttm_real                                                     0
natixis_commercial_mortgage_security_4                                 0
natixis_commercial_mortgage_security_3                                 0
natixis_asg_managed_futures_strategy_y                                 0
natixis_asg_global_alternatives_y                                      0
nationwide_wcm_focused_sm_cp_instl_svc                                 0
nationwide_small_cap_index_r6                                          0
nationwide_mutual_insurance_company_9                                  0
nationwide_mutual_insurance_company_4                                  0
nationwide_mutual_insurance_company_0                                  0
nationwide_multi                                                       0
nationwide_mid_cap_market_idx_r6                                       0
nationwide_loomis_short_term_bd_r6                                     0
nationwide_loomis_all_cap_gr_r6                    

In [85]:
df1.fillna(0, inplace=True)

In [55]:
df1.drop(columns={'bond_number_of_holdings'}, inplace=True)


In [56]:
df1[df1['region_exposure_americas'].isna()]

,div_yield_ttm_real,30_day_sec_yield_real,weighted_avg_pe_ratio_real,weighted_avg_price_to_sales_ratio,weighted_avg_price_to_book_ratio,weighted_median_ROE,weighted_median_roa,forecasted_div_yield_real,forecasted_pe_ratio,forecasted_price_to_sales_ratio,forecasted_price_to_book_ratio,nu_of_holdings,bonds_current_yield,bond_yield_to_maturity,effective_duration,avg_coupon,avg_credit_score_1_best,bond_average_price,forecasted_5yr_earnings_growth,forecasted_book_value_growth,forecasted_cash_flow_growth,forecasted_earnings_growth,forecasted_revenue_growth,region_exposure_americas,region_exposure_north_america,region_exposure_latin_america,region_exposure_greater_europe,region_exposure_uk,region_exposure_europe_developed,region_exposure_europe_emerging,region_exposure_africa_middle_east,region_exposure_greater_asia,region_exposure_japan,region_exposure_australiasia,region_exposure_asia_developed,region_exposure_asia_emerging,region_exposure_developed_markets,region_exposure_emerging_markets,stock_sector_exp_basic_materials,stock_sector_exp_communication_service,stock_sector_exp_consumer_cycl,stock_sector_exp_cons_defensive,stock_sector_exp_energy,stock_sector_exp_fin_serv,stock_sector_exp_healthcare,stock_sector_exp_industrial,stock_sector_exp_real_estate,stock_sector_exp_tech,stock_sector_exp_utilities,stock_market_cap_giant,stock_market_cap_large,stock_market_cap_medium,stock_market_cap_small,stock_market_cap_micro,risk_beta,risk_sharpe_ratio,risk_sortino_ratio,risk_daily_VAR,risk_monthly_VAR,stock_style_exp_lcvalue,stock_style_exp_lcblend,stock_style_exp_lcgrowth,stock_style_exp_mcvalue,stock_style_exp_mcblend,stock_style_exp_bcgrowth,stock_style_exp_scvalue,stock_style_exp_smblend,stock_style_exp_smgrowth,open_to_new_investors_y,min_initial_investment,fee_actual_mgmt_fee,index_fund_y,leverage_y,fund_of_funds_y,fund_currency_hedged_y,fund_socially_responsible_y,fund_available_for_insurance_place_y,total_return_1m,total_return_3m,total_return_6m,total_return_ytd,total_return_1y,total_return_3y,total_return_5y,total_return_10y,total_return_15y,total_return_20y,bond_quality_AAA,bond_quality_AA,bond_quality_A,bond_quality_BBB,bond_quality_BB,bond_quality_B,bond_quality_below_B,position_cash_net,position_cash_long,position_cash_short,position_stock_net,position_stock_long,position_stock_short,position_bond_net,position_bond_long,position_bond_short,position_convertible_net,position_convertible_long,position_convertible_short,position_preferred_net,position_preferred_long,position_preferred_short,position_other_net,position_other_long,position_other_short,expense_ratio,aum,share_aum,net_asset_value,turnover_ratio,fund_flows,risk_score_low_bond_quality,risk_score_below_B_over_15p,risk_score_yield_2_maturity_low,risk_score_duration_high,risk_score_preferred_net_over25p,stock_sector_rising_interest_rate_bad,risk_score_bad_equities_rising_rates,risk_score_avg_price_book_high,risk_score_avg_price_earnings_high,risk_score_smallest_companies,risk_score_smallest_companies_over50,risk_score_emerging_markets_over_45,risk_score_energy_over_15,risk_total,risk_score_PE_expensive,risk_score_PS_expensive,risk_score_low_ROE,risk_score_duration_long,risk_score_low_bond_yields,risk_score_risky_bonds,risk_score_latam,00,000,0000,00000,000000_maturity_20250612,000000_maturity_20250920,000000_maturity_20260126,000000_maturity_20281115,00000_,00000_0,00000_02,00000_03,00000_06,00000_07,00000_08,00000_09,00000_10yr_tnotes_dec9,00000_11,00000_112495,00000_112498,00000_112501_fee_ccpcdx,00000_112504_fee_ccpcdx,00000_112505_fee_ccpitraxx,00000_112507_fee_ccpcdx,00000_112513_fee_ccpcdx,00000_112519_fee_ccpcdx,00000_112524_fee_ccpcdx,00000_12,00000_1_ccpcdx,00000_1_ccpitraxx,00000_2_ccpcdx,00000_2i65brsb7_fee_ccpcdx,00000_2i65bybd6_fee_ccpcdx,00000_2i65bzdm1_fee_ccpcdx,00000_558600990_fee_ccpcdx,00000_96569886_fee_ccpitraxx,00000_99d0b4bz9_ccpvanilla,00000_cdx_ccp_ice,00000_cdxig033_fee_ccpcdx,00000_cdxnahy_500bp_ccp,00000_cdxnaig_100bp_ccp,00000_cosp9u3205_fee_c

In [57]:
df1.dropna(subset= ['region_exposure_japan'], inplace=True)

In [50]:
df1[df1['turnover_ratio'].isna()].head()

,div_yield_ttm_real,30_day_sec_yield_real,weighted_avg_pe_ratio_real,weighted_avg_price_to_sales_ratio,weighted_avg_price_to_book_ratio,weighted_median_ROE,weighted_median_roa,forecasted_div_yield_real,forecasted_pe_ratio,forecasted_price_to_sales_ratio,forecasted_price_to_book_ratio,nu_of_holdings,forecasted_ps_ratio,stock_forecasted_price_to_book_value,stock_number_of_stock_holdings,bonds_current_yield,bond_yield_to_maturity,effective_duration,avg_coupon,avg_credit_score_1_best,bond_average_price,bond_number_of_holdings,forecasted_5yr_earnings_growth,forecasted_book_value_growth,forecasted_cash_flow_growth,forecasted_earnings_growth,forecasted_revenue_growth,region_exposure_americas,region_exposure_north_america,region_exposure_latin_america,region_exposure_greater_europe,region_exposure_uk,region_exposure_europe_developed,region_exposure_europe_emerging,region_exposure_africa_middle_east,region_exposure_greater_asia,region_exposure_japan,region_exposure_australiasia,region_exposure_asia_developed,region_exposure_asia_emerging,region_exposure_developed_markets,region_exposure_emerging_markets,stock_sector_exp_basic_materials,stock_sector_exp_communication_service,stock_sector_exp_consumer_cycl,stock_sector_exp_cons_defensive,stock_sector_exp_energy,stock_sector_exp_fin_serv,stock_sector_exp_healthcare,stock_sector_exp_industrial,stock_sector_exp_real_estate,stock_sector_exp_tech,stock_sector_exp_utilities,stock_market_cap_giant,stock_market_cap_large,stock_market_cap_medium,stock_market_cap_small,stock_market_cap_micro,risk_beta,risk_sharpe_ratio,risk_sortino_ratio,risk_daily_VAR,risk_monthly_VAR,stock_style_exp_lcvalue,stock_style_exp_lcblend,stock_style_exp_lcgrowth,stock_style_exp_mcvalue,stock_style_exp_mcblend,stock_style_exp_bcgrowth,stock_style_exp_scvalue,stock_style_exp_smblend,stock_style_exp_smgrowth,open_to_new_investors_y,min_initial_investment,fee_actual_mgmt_fee,fee_admin_fee,fee_performance_fee,fee_max_deferred_load,fee_max_redemption,index_fund_y,leverage_y,fund_of_funds_y,fund_currency_hedged_y,fund_socially_responsible_y,fund_available_for_insurance_place_y,total_return_1m,total_return_3m,total_return_6m,total_return_ytd,total_return_1y,total_return_3y,total_return_5y,total_return_10y,total_return_15y,total_return_20y,bond_quality_AAA,bond_quality_AA,bond_quality_A,bond_quality_BBB,bond_quality_BB,bond_quality_B,bond_quality_below_B,position_cash_net,position_cash_long,position_cash_short,position_stock_net,position_stock_long,position_stock_short,position_bond_net,position_bond_long,position_bond_short,position_convertible_net,position_convertible_long,position_convertible_short,position_preferred_net,position_preferred_long,position_preferred_short,position_other_net,position_other_long,position_other_short,expense_ratio,aum,share_aum,net_asset_value,turnover_ratio,fund_flows,risk_score_low_bond_quality,risk_score_below_B_over_15p,risk_score_yield_2_maturity_low,risk_score_duration_high,risk_score_preferred_net_over25p,stock_sector_rising_interest_rate_bad,risk_score_bad_equities_rising_rates,risk_score_avg_price_book_high,risk_score_avg_price_earnings_high,risk_score_smallest_companies,risk_score_smallest_companies_over50,risk_score_emerging_markets_over_45,risk_score_energy_over_15,risk_total,risk_score_PE_expensive,risk_score_PS_expensive,risk_score_low_ROE,risk_score_duration_long,risk_score_low_bond_yields,risk_score_risky_bonds,risk_score_latam,00,000,0000,00000,000000_maturity_20250612,000000_maturity_20250920,000000_maturity_20260126,000000_maturity_20281115,00000_,00000_0,00000_02,00000_03,00000_06,00000_07,00000_08,00000_09,00000_10yr_tnotes_dec9,00000_11,00000_112495,00000_112498,00000_112501_fee_ccpcdx,00000_112504_fee_ccpcdx,00000_112505_fee_ccpitraxx,00000_112507_fee_ccpcdx,00000_112513_fee_ccpcdx,00000_112519_fee_ccpcdx,00000_112524_fee_ccpcdx,00000_12,00000_1_ccpcdx,00000_1_ccpitraxx,00000_2_ccpcdx,00000_2i65brsb7_fee_ccpcdx,00000_2i65bybd6_fee_ccpcdx,00000_2i65bzdm1_fee_ccpcdx,00000_5586

In [126]:
df1.reset_index(inplace=True)

In [121]:
df0 = pd.read_csv('zacks_full_banana.csv')

In [131]:
df0 = deepcopy(df0[['Fund Name', 'Ticker']])

In [134]:
df0.rename(columns= {'Fund Name': 'fund_name',  'Ticker':'ticker'}, inplace=True)

In [127]:
df1.head()

,ticker,div_yield_ttm_real,30_day_sec_yield_real,weighted_avg_pe_ratio_real,weighted_avg_price_to_sales_ratio,weighted_avg_price_to_book_ratio,weighted_median_ROE,weighted_median_roa,forecasted_div_yield_real,forecasted_pe_ratio,forecasted_price_to_sales_ratio,forecasted_price_to_book_ratio,nu_of_holdings,bonds_current_yield,bond_yield_to_maturity,effective_duration,avg_coupon,avg_credit_score_1_best,bond_average_price,forecasted_5yr_earnings_growth,forecasted_book_value_growth,forecasted_cash_flow_growth,forecasted_earnings_growth,forecasted_revenue_growth,region_exposure_americas,region_exposure_north_america,region_exposure_latin_america,region_exposure_greater_europe,region_exposure_uk,region_exposure_europe_developed,region_exposure_europe_emerging,region_exposure_africa_middle_east,region_exposure_greater_asia,region_exposure_japan,region_exposure_australiasia,region_exposure_asia_developed,region_exposure_asia_emerging,region_exposure_developed_markets,region_exposure_emerging_markets,stock_sector_exp_basic_materials,stock_sector_exp_communication_service,stock_sector_exp_consumer_cycl,stock_sector_exp_cons_defensive,stock_sector_exp_energy,stock_sector_exp_fin_serv,stock_sector_exp_healthcare,stock_sector_exp_industrial,stock_sector_exp_real_estate,stock_sector_exp_tech,stock_sector_exp_utilities,stock_market_cap_giant,stock_market_cap_large,stock_market_cap_medium,stock_market_cap_small,stock_market_cap_micro,risk_beta,risk_sharpe_ratio,risk_sortino_ratio,risk_daily_VAR,risk_monthly_VAR,stock_style_exp_lcvalue,stock_style_exp_lcblend,stock_style_exp_lcgrowth,stock_style_exp_mcvalue,stock_style_exp_mcblend,stock_style_exp_bcgrowth,stock_style_exp_scvalue,stock_style_exp_smblend,stock_style_exp_smgrowth,open_to_new_investors_y,min_initial_investment,fee_actual_mgmt_fee,index_fund_y,leverage_y,fund_of_funds_y,fund_currency_hedged_y,fund_socially_responsible_y,fund_available_for_insurance_place_y,total_return_1m,total_return_3m,total_return_6m,total_return_ytd,total_return_1y,total_return_3y,total_return_5y,total_return_10y,total_return_15y,total_return_20y,bond_quality_AAA,bond_quality_AA,bond_quality_A,bond_quality_BBB,bond_quality_BB,bond_quality_B,bond_quality_below_B,position_cash_net,position_cash_long,position_cash_short,position_stock_net,position_stock_long,position_stock_short,position_bond_net,position_bond_long,position_bond_short,position_convertible_net,position_convertible_long,position_convertible_short,position_preferred_net,position_preferred_long,position_preferred_short,position_other_net,position_other_long,position_other_short,expense_ratio,aum,share_aum,net_asset_value,turnover_ratio,fund_flows,risk_score_low_bond_quality,risk_score_below_B_over_15p,risk_score_yield_2_maturity_low,risk_score_duration_high,risk_score_preferred_net_over25p,stock_sector_rising_interest_rate_bad,risk_score_bad_equities_rising_rates,risk_score_avg_price_book_high,risk_score_avg_price_earnings_high,risk_score_smallest_companies,risk_score_smallest_companies_over50,risk_score_emerging_markets_over_45,risk_score_energy_over_15,risk_total,risk_score_PE_expensive,risk_score_PS_expensive,risk_score_low_ROE,risk_score_duration_long,risk_score_low_bond_yields,risk_score_risky_bonds,risk_score_latam,00,000,0000,00000,000000_maturity_20250612,000000_maturity_20250920,000000_maturity_20260126,000000_maturity_20281115,00000_,00000_0,00000_02,00000_03,00000_06,00000_07,00000_08,00000_09,00000_10yr_tnotes_dec9,00000_11,00000_112495,00000_112498,00000_112501_fee_ccpcdx,00000_112504_fee_ccpcdx,00000_112505_fee_ccpitraxx,00000_112507_fee_ccpcdx,00000_112513_fee_ccpcdx,00000_112519_fee_ccpcdx,00000_112524_fee_ccpcdx,00000_12,00000_1_ccpcdx,00000_1_ccpitraxx,00000_2_ccpcdx,00000_2i65brsb7_fee_ccpcdx,00000_2i65bybd6_fee_ccpcdx,00000_2i65bzdm1_fee_ccpcdx,00000_558600990_fee_ccpcdx,00000_96569886_fee_ccpitraxx,00000_99d0b4bz9_ccpvanilla,00000_cdx_ccp_ice,00000_cdxig033_fee_ccpcdx,00000_cdxnahy_500bp_ccp,00000_cdxnaig_100bp_ccp,00000_cosp9u320

In [136]:
df2 = pd.merge(df0, df1, how='inner',on='ticker')

In [138]:
df2.set_index('ticker', inplace=True)

In [176]:
df2.shape

(14159, 24118)

In [174]:
df2.to_csv('final_df2.csv')

In [175]:
from annoy import AnnoyIndex

In [ ]:
f = 40
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for i in range(1000):
    v = [random.gauss(0, 1) for z in range(f)]
    t.add_item(i, v)

t.build(10) # 10 trees
t.save('test.ann')

# ...

u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(0, 1000)) # will find the 1000 nearest neighbors

In [179]:
df2.head()

,fund_name,div_yield_ttm_real,30_day_sec_yield_real,weighted_avg_pe_ratio_real,weighted_avg_price_to_sales_ratio,weighted_avg_price_to_book_ratio,weighted_median_ROE,weighted_median_roa,forecasted_div_yield_real,forecasted_pe_ratio,forecasted_price_to_sales_ratio,forecasted_price_to_book_ratio,nu_of_holdings,bonds_current_yield,bond_yield_to_maturity,effective_duration,avg_coupon,avg_credit_score_1_best,bond_average_price,forecasted_5yr_earnings_growth,forecasted_book_value_growth,forecasted_cash_flow_growth,forecasted_earnings_growth,forecasted_revenue_growth,region_exposure_americas,region_exposure_north_america,region_exposure_latin_america,region_exposure_greater_europe,region_exposure_uk,region_exposure_europe_developed,region_exposure_europe_emerging,region_exposure_africa_middle_east,region_exposure_greater_asia,region_exposure_japan,region_exposure_australiasia,region_exposure_asia_developed,region_exposure_asia_emerging,region_exposure_developed_markets,region_exposure_emerging_markets,stock_sector_exp_basic_materials,stock_sector_exp_communication_service,stock_sector_exp_consumer_cycl,stock_sector_exp_cons_defensive,stock_sector_exp_energy,stock_sector_exp_fin_serv,stock_sector_exp_healthcare,stock_sector_exp_industrial,stock_sector_exp_real_estate,stock_sector_exp_tech,stock_sector_exp_utilities,stock_market_cap_giant,stock_market_cap_large,stock_market_cap_medium,stock_market_cap_small,stock_market_cap_micro,risk_beta,risk_sharpe_ratio,risk_sortino_ratio,risk_daily_VAR,risk_monthly_VAR,stock_style_exp_lcvalue,stock_style_exp_lcblend,stock_style_exp_lcgrowth,stock_style_exp_mcvalue,stock_style_exp_mcblend,stock_style_exp_bcgrowth,stock_style_exp_scvalue,stock_style_exp_smblend,stock_style_exp_smgrowth,open_to_new_investors_y,min_initial_investment,fee_actual_mgmt_fee,index_fund_y,leverage_y,fund_of_funds_y,fund_currency_hedged_y,fund_socially_responsible_y,fund_available_for_insurance_place_y,total_return_1m,total_return_3m,total_return_6m,total_return_ytd,total_return_1y,total_return_3y,total_return_5y,total_return_10y,total_return_15y,total_return_20y,bond_quality_AAA,bond_quality_AA,bond_quality_A,bond_quality_BBB,bond_quality_BB,bond_quality_B,bond_quality_below_B,position_cash_net,position_cash_long,position_cash_short,position_stock_net,position_stock_long,position_stock_short,position_bond_net,position_bond_long,position_bond_short,position_convertible_net,position_convertible_long,position_convertible_short,position_preferred_net,position_preferred_long,position_preferred_short,position_other_net,position_other_long,position_other_short,expense_ratio,aum,share_aum,net_asset_value,turnover_ratio,fund_flows,risk_score_low_bond_quality,risk_score_below_B_over_15p,risk_score_yield_2_maturity_low,risk_score_duration_high,risk_score_preferred_net_over25p,stock_sector_rising_interest_rate_bad,risk_score_bad_equities_rising_rates,risk_score_avg_price_book_high,risk_score_avg_price_earnings_high,risk_score_smallest_companies,risk_score_smallest_companies_over50,risk_score_emerging_markets_over_45,risk_score_energy_over_15,risk_total,risk_score_PE_expensive,risk_score_PS_expensive,risk_score_low_ROE,risk_score_duration_long,risk_score_low_bond_yields,risk_score_risky_bonds,risk_score_latam,00,000,0000,00000,000000_maturity_20250612,000000_maturity_20250920,000000_maturity_20260126,000000_maturity_20281115,00000_,00000_0,00000_02,00000_03,00000_06,00000_07,00000_08,00000_09,00000_10yr_tnotes_dec9,00000_11,00000_112495,00000_112498,00000_112501_fee_ccpcdx,00000_112504_fee_ccpcdx,00000_112505_fee_ccpitraxx,00000_112507_fee_ccpcdx,00000_112513_fee_ccpcdx,00000_112519_fee_ccpcdx,00000_112524_fee_ccpcdx,00000_12,00000_1_ccpcdx,00000_1_ccpitraxx,00000_2_ccpcdx,00000_2i65brsb7_fee_ccpcdx,00000_2i65bybd6_fee_ccpcdx,00000_2i65bzdm1_fee_ccpcdx,00000_558600990_fee_ccpcdx,00000_96569886_fee_ccpitraxx,00000_99d0b4bz9_ccpvanilla,00000_cdx_ccp_ice,00000_cdxig033_fee_ccpcdx,00000_cdxnahy_500bp_ccp,00000_cdxnaig_100bp_ccp,00000_cosp9u

In [178]:
df2[['fund_name', 'div_yield_ttm_real', '30_day_sec_yield_real', 'weighted_avg_pe_ratio_real', 'weighted_avg_price_to_sales_ratio', 'weighted_avg_price_to_book_ratio', 'weighted_median_ROE', 'weighted_median_roa', 'forecasted_div_yield_real', 'forecasted_pe_ratio', 'forecasted_price_to_sales_ratio', 'forecasted_price_to_book_ratio', 'nu_of_holdings', 'bonds_current_yield', 'bond_yield_to_maturity', 'effective_duration', 'avg_coupon', 'avg_credit_score_1_best', 'bond_average_price', 'forecasted_5yr_earnings_growth', 'forecasted_book_value_growth', 'forecasted_cash_flow_growth', 'forecasted_earnings_growth', 'forecasted_revenue_growth', 'region_exposure_americas', 'region_exposure_north_america', 'region_exposure_latin_america', 'region_exposure_greater_europe', 'region_exposure_uk', 'region_exposure_europe_developed', 'region_exposure_europe_emerging', 'region_exposure_africa_middle_east', 'region_exposure_greater_asia', 'region_exposure_japan', 'region_exposure_australiasia', 'region_exposure_asia_developed', 'region_exposure_asia_emerging', 'region_exposure_developed_markets', 'region_exposure_emerging_markets', 'stock_sector_exp_basic_materials', 'stock_sector_exp_communication_service', 'stock_sector_exp_consumer_cycl', 'stock_sector_exp_cons_defensive', 'stock_sector_exp_energy', 'stock_sector_exp_fin_serv', 'stock_sector_exp_healthcare', 'stock_sector_exp_industrial', 'stock_sector_exp_real_estate', 'stock_sector_exp_tech', 'stock_sector_exp_utilities', 'stock_market_cap_giant', 'stock_market_cap_large', 'stock_market_cap_medium', 'stock_market_cap_small', 'stock_market_cap_micro', 'risk_beta', 'risk_sharpe_ratio', 'risk_sortino_ratio', 'risk_daily_VAR', 'risk_monthly_VAR', 'stock_style_exp_lcvalue', 'stock_style_exp_lcblend', 'stock_style_exp_lcgrowth', 'stock_style_exp_mcvalue', 'stock_style_exp_mcblend', 'stock_style_exp_bcgrowth', 'stock_style_exp_scvalue', 'stock_style_exp_smblend', 'stock_style_exp_smgrowth', 'open_to_new_investors_y', 'min_initial_investment', 'fee_actual_mgmt_fee', 'index_fund_y', 'leverage_y', 'fund_of_funds_y', 'fund_currency_hedged_y', 'fund_socially_responsible_y', 'fund_available_for_insurance_place_y', 'total_return_1m', 'total_return_3m', 'total_return_6m', 'total_return_ytd', 'total_return_1y', 'total_return_3y', 'total_return_5y', 'total_return_10y', 'total_return_15y', 'total_return_20y', 'bond_quality_AAA', 'bond_quality_AA', 'bond_quality_A', 'bond_quality_BBB', 'bond_quality_BB', 'bond_quality_B', 'bond_quality_below_B', 'position_cash_net', 'position_cash_long', 'position_cash_short', 'position_stock_net', 'position_stock_long', 'position_stock_short', 'position_bond_net', 'position_bond_long', 'position_bond_short', 'position_convertible_net', 'position_convertible_long', 'position_convertible_short', 'position_preferred_net', 'position_preferred_long', 'position_preferred_short', 'position_other_net', 'position_other_long', 'position_other_short', 'expense_ratio', 'aum', 'share_aum', 'net_asset_value', 'turnover_ratio', 'fund_flows', 'risk_score_low_bond_quality', 'risk_score_below_B_over_15p', 'risk_score_yield_2_maturity_low', 'risk_score_duration_high', 'risk_score_preferred_net_over25p', 'stock_sector_rising_interest_rate_bad', 'risk_score_bad_equities_rising_rates', 'risk_score_avg_price_book_high', 'risk_score_avg_price_earnings_high', 'risk_score_smallest_companies', 'risk_score_smallest_companies_over50', 'risk_score_emerging_markets_over_45', 'risk_score_energy_over_15', 'risk_total', 'risk_score_PE_expensive', 'risk_score_PS_expensive', 'risk_score_low_ROE', 'risk_score_duration_long', 'risk_score_low_bond_yields', 'risk_score_risky_bonds', 'risk_score_latam']]

,fund_name,div_yield_ttm_real,30_day_sec_yield_real,weighted_avg_pe_ratio_real,weighted_avg_price_to_sales_ratio,weighted_avg_price_to_book_ratio,weighted_median_ROE,weighted_median_roa,forecasted_div_yield_real,forecasted_pe_ratio,forecasted_price_to_sales_ratio,forecasted_price_to_book_ratio,nu_of_holdings,bonds_current_yield,bond_yield_to_maturity,effective_duration,avg_coupon,avg_credit_score_1_best,bond_average_price,forecasted_5yr_earnings_growth,forecasted_book_value_growth,forecasted_cash_flow_growth,forecasted_earnings_growth,forecasted_revenue_growth,region_exposure_americas,region_exposure_north_america,region_exposure_latin_america,region_exposure_greater_europe,region_exposure_uk,region_exposure_europe_developed,region_exposure_europe_emerging,region_exposure_africa_middle_east,region_exposure_greater_asia,region_exposure_japan,region_exposure_australiasia,region_exposure_asia_developed,region_exposure_asia_emerging,region_exposure_developed_markets,region_exposure_emerging_markets,stock_sector_exp_basic_materials,stock_sector_exp_communication_service,stock_sector_exp_consumer_cycl,stock_sector_exp_cons_defensive,stock_sector_exp_energy,stock_sector_exp_fin_serv,stock_sector_exp_healthcare,stock_sector_exp_industrial,stock_sector_exp_real_estate,stock_sector_exp_tech,stock_sector_exp_utilities,stock_market_cap_giant,stock_market_cap_large,stock_market_cap_medium,stock_market_cap_small,stock_market_cap_micro,risk_beta,risk_sharpe_ratio,risk_sortino_ratio,risk_daily_VAR,risk_monthly_VAR,stock_style_exp_lcvalue,stock_style_exp_lcblend,stock_style_exp_lcgrowth,stock_style_exp_mcvalue,stock_style_exp_mcblend,stock_style_exp_bcgrowth,stock_style_exp_scvalue,stock_style_exp_smblend,stock_style_exp_smgrowth,open_to_new_investors_y,min_initial_investment,fee_actual_mgmt_fee,index_fund_y,leverage_y,fund_of_funds_y,fund_currency_hedged_y,fund_socially_responsible_y,fund_available_for_insurance_place_y,total_return_1m,total_return_3m,total_return_6m,total_return_ytd,total_return_1y,total_return_3y,total_return_5y,total_return_10y,total_return_15y,total_return_20y,bond_quality_AAA,bond_quality_AA,bond_quality_A,bond_quality_BBB,bond_quality_BB,bond_quality_B,bond_quality_below_B,position_cash_net,position_cash_long,position_cash_short,position_stock_net,position_stock_long,position_stock_short,position_bond_net,position_bond_long,position_bond_short,position_convertible_net,position_convertible_long,position_convertible_short,position_preferred_net,position_preferred_long,position_preferred_short,position_other_net,position_other_long,position_other_short,expense_ratio,aum,share_aum,net_asset_value,turnover_ratio,fund_flows,risk_score_low_bond_quality,risk_score_below_B_over_15p,risk_score_yield_2_maturity_low,risk_score_duration_high,risk_score_preferred_net_over25p,stock_sector_rising_interest_rate_bad,risk_score_bad_equities_rising_rates,risk_score_avg_price_book_high,risk_score_avg_price_earnings_high,risk_score_smallest_companies,risk_score_smallest_companies_over50,risk_score_emerging_markets_over_45,risk_score_energy_over_15,risk_total,risk_score_PE_expensive,risk_score_PS_expensive,risk_score_low_ROE,risk_score_duration_long,risk_score_low_bond_yields,risk_score_risky_bonds,risk_score_latam
ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAADX,Aberdeen Income Builder A,2.64,1.87,19.770,3.2560,3.323,22.54,7.50,1.88,18.090,0.0000,0.0000,112.0,5.97,5.50,0.000,6.19,0.0,103.88,0.00,0.00,0.00,0.00,0.00,96.82,96.05,0.77,1.83,0.00,1.83,0.00,0.00,1.35,0.00,0.00,0.00,1.35,98.65,1.35,0.00,7.14,5.70,6.40,0.00,20.58,12.37,4.21,4.25,33.59,5.76,62.67,34.57,2.76,0.00,0.00,0.8260,1.0620,1.3440,2.13,7.35,29.90,37.34,30.12,0.00,2.65,0.00,0.00,0.00,0.00,1,1000.0,1.00,0,0,0,0,0,0,1.73,5.17,6.49,17.97,13.78,10.55,7.31,10.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.18,3.18,0.00,45.66,45.66,0.00,50.38,50.38,0.00,0.00,0.00,0.0,0.79,0.79,0.

In [185]:
df['holdings']

ticker
VFIAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
FXAIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VGTSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VITSX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSMX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTPSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VIIIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VINIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VBTLX    ['federal_national_mortgage_association_3%', '...
FCNTX    ['facebook_inc_a', 'amazon.com_inc', 'berkshir...
VWENX    ['microsoft_corp', 'verizon_communications_inc...
VTIAX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
DODGX    ['wells_fargo_&_co', 'charles_schwab_corp', 'o...
PIMIX    ['irs_usd_3.00000_06/19/19-5y_cme', 'irs_jpy_0...
RERGX    ['airbus_se', 'aia_group_ltd', 'nintendo_co_lt...
VWIUX    ['southeast_ala_gas_supply_dist_gas_supp

In [ ]:
[re.sub(r"\d.?\d*\%","bond",x) for x in new_holdings]

In [183]:
import re

In [ ]:
" ".join(x['holdings']) 

In [192]:
df['holdings']= deepcopy(df['holdings'].apply(lambda x: re.sub(r"\d.?\d*\%","bond",x)))

In [193]:
df['holdings'] = deepcopy(df['holdings'].apply(lambda x: re.sub(r"\d+\/\d+\/\d{2,}","date",x)))

In [194]:
df['holdings'] = deepcopy(df['holdings'].apply(lambda x: re.sub(r"\d{3,}","",x)))

In [195]:
df['holdings']

ticker
VFIAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
FXAIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VGTSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VITSX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSMX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTPSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VIIIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VINIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VBTLX    ['federal_national_mortgage_association_bond',...
FCNTX    ['facebook_inc_a', 'amazon.com_inc', 'berkshir...
VWENX    ['microsoft_corp', 'verizon_communications_inc...
VTIAX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
DODGX    ['wells_fargo_&_co', 'charles_schwab_corp', 'o...
PIMIX    ['irs_usd_3._date-5y_cme', 'irs_jpy_0._date-10...
RERGX    ['airbus_se', 'aia_group_ltd', 'nintendo_co_lt...
VWIUX    ['southeast_ala_gas_supply_dist_gas_supp

In [197]:
df.holdings

ticker
VFIAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
FXAIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VGTSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VITSX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSMX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTPSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VIIIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VINIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VBTLX    ['federal_national_mortgage_association_bond',...
FCNTX    ['facebook_inc_a', 'amazon.com_inc', 'berkshir...
VWENX    ['microsoft_corp', 'verizon_communications_inc...
VTIAX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
DODGX    ['wells_fargo_&_co', 'charles_schwab_corp', 'o...
PIMIX    ['irs_usd_3._date-5y_cme', 'irs_jpy_0._date-10...
RERGX    ['airbus_se', 'aia_group_ltd', 'nintendo_co_lt...
VWIUX    ['southeast_ala_gas_supply_dist_gas_supp

In [230]:
all_holdings = []
for i in tqdm(df['holdings']):
    i.to_list()
#     hold = []
#     print(type(i))
#     print(i, '\n')
#     for j in i:
#         print(j)
#         hold.append(j)
#     all_holdings.append(hold)

AttributeError: 'str' object has no attribute 'to_list'

In [219]:
all_holdings[0]

['[',
 "'",
 'm',
 'i',
 'c',
 'r',
 'o',
 's',
 'o',
 'f',
 't',
 '_',
 'c',
 'o',
 'r',
 'p',
 "'",
 ',',
 ' ',
 "'",
 'a',
 'p',
 'p',
 'l',
 'e',
 '_',
 'i',
 'n',
 'c',
 "'",
 ',',
 ' ',
 "'",
 'a',
 'm',
 'a',
 'z',
 'o',
 'n',
 '.',
 'c',
 'o',
 'm',
 '_',
 'i',
 'n',
 'c',
 "'",
 ',',
 ' ',
 "'",
 'f',
 'a',
 'c',
 'e',
 'b',
 'o',
 'o',
 'k',
 '_',
 'i',
 'n',
 'c',
 '_',
 'a',
 "'",
 ',',
 ' ',
 "'",
 'b',
 'e',
 'r',
 'k',
 's',
 'h',
 'i',
 'r',
 'e',
 '_',
 'h',
 'a',
 't',
 'h',
 'a',
 'w',
 'a',
 'y',
 '_',
 'i',
 'n',
 'c',
 '_',
 'b',
 "'",
 ',',
 ' ',
 "'",
 'j',
 'p',
 'm',
 'o',
 'r',
 'g',
 'a',
 'n',
 '_',
 'c',
 'h',
 'a',
 's',
 'e',
 '_',
 '&',
 '_',
 'c',
 'o',
 "'",
 ',',
 ' ',
 "'",
 'a',
 'l',
 'p',
 'h',
 'a',
 'b',
 'e',
 't',
 '_',
 'i',
 'n',
 'c',
 '_',
 'c',
 'l',
 'a',
 's',
 's',
 '_',
 'c',
 "'",
 ',',
 ' ',
 "'",
 'a',
 'l',
 'p',
 'h',
 'a',
 'b',
 'e',
 't',
 '_',
 'i',
 'n',
 'c',
 '_',
 'a',
 "'",
 ',',
 ' ',
 "'",
 'j',
 'o',
 'h',
 'n',
 's'

In [203]:
df['holdings']

ticker
VFIAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSAX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
FXAIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VGTSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VITSX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTSMX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VTPSX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
VIIIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VINIX    ['microsoft_corp', 'apple_inc', 'amazon.com_in...
VBTLX    ['federal_national_mortgage_association_bond',...
FCNTX    ['facebook_inc_a', 'amazon.com_inc', 'berkshir...
VWENX    ['microsoft_corp', 'verizon_communications_inc...
VTIAX    ['nestle_sa', 'alibaba_group_holding_ltd_adr',...
DODGX    ['wells_fargo_&_co', 'charles_schwab_corp', 'o...
PIMIX    ['irs_usd_3._date-5y_cme', 'irs_jpy_0._date-10...
RERGX    ['airbus_se', 'aia_group_ltd', 'nintendo_co_lt...
VWIUX    ['southeast_ala_gas_supply_dist_gas_supp

In [202]:
all_holdings

["['microsoft_corp', 'apple_inc', 'amazon.com_inc', 'facebook_inc_a', 'berkshire_hathaway_inc_b', 'jpmorgan_chase_&_co', 'alphabet_inc_class_c', 'alphabet_inc_a', 'johnson_&_johnson', 'procter_&_gamble_co', 'visa_inc_class_a', 'exxon_mobil_corp', 'at&t_inc', 'bank_of_america_corp', 'the_home_depot_inc', 'intel_corp', 'verizon_communications_inc', 'mastercard_inc_a', 'the_walt_disney_co', 'unitedhealth_group_inc', 'merck_&_co_inc', 'chevron_corp', 'pfizer_inc', 'coca-cola_co', 'wells_fargo_&_co']",
 "['microsoft_corp', 'apple_inc', 'amazon.com_inc', 'facebook_inc_a', 'berkshire_hathaway_inc_b', 'jpmorgan_chase_&_co', 'alphabet_inc_a', 'alphabet_inc_class_c', 'johnson_&_johnson', 'procter_&_gamble_co', 'visa_inc_class_a', 'exxon_mobil_corp', 'at&t_inc', 'bank_of_america_corp', 'the_home_depot_inc', 'intel_corp', 'verizon_communications_inc', 'mastercard_inc_a', 'the_walt_disney_co', 'unitedhealth_group_inc', 'merck_&_co_inc', 'chevron_corp', 'pfizer_inc', 'coca-cola_co', 'wells_fargo_&_c

In [ ]:
df['holdings2'] = all_holdings